<a href="https://colab.research.google.com/github/Igirisu861/Parcial1_Ciberseguridad_DianaRuiz/blob/main/Parcial1_Ejercicio1_DianaRuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Parcial 1**

## **Ejercicio 1**

El alumno deberá utilizar el algoritmo RSA para que Alice le mande un mensaje
cifrado a Bob con las siguientes características.
El mensaje “M” será de 1050 caracteres.

* El alumno deberá generar h(M) para una vez descifrado constatar la
 autenticidad del mensaje M.
* Por cuestiones de eficiencia el alumno deberá dividir en partes más
pequeñas de 128 caracteres el mensaje antes de ser cifrado y enviado.
* Alice cifrará los mensajes producto de la división del mensaje original con la
llave pública de Bob.
* Bob descifrará los mensajes con su llave privada y con cada uno de ellos
obtendrá el mensaje original de 1050 caracteres.
* Bob generará el hash del mensaje recibido h(M’).
* Debemos comparar si h(M) = h(M’)

In [100]:
!pip install pycryptodome

In [101]:
import Crypto.Util.number
import random
import hashlib
import random
import string

### **Creación de las llaves de Alice y Bob**

In [102]:
e = 65537

#llave pública de Alice
pA = Crypto.Util.number.getPrime(1024)
qA = Crypto.Util.number.getPrime(1024)
nA = pA * qA

#Llave privada de Alice
phiA = (pA - 1) * (qA - 1)
dA = pow(e, -1, phiA)


#llave pública de Bob
pB = Crypto.Util.number.getPrime(1024)
qB = Crypto.Util.number.getPrime(1024)
nB = pB * qB

#Llave privada de Bob
phiB = (pB - 1) * (qB - 1)
dB = pow(e, -1, phiB)

### **Generación del mensaje de 1050 caracteres**

In [103]:
#generación del mensaje de 1050 caracteres
def generate_message(length):
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(length))

msg = generate_message(1050)
msg

'kxlkmugukrfpnkanzaacxklfvqifgamefkywjaepgksgulzirezjpncebivuhljziqxwiwtigqwnuvfbmrxfnbnventsrkhbjzvmiidlvmzbruzhdzoneaufkquorvmlsokpmxinaipetdlbraynquimrxsarsszzblpdgvrmaljgbxuqdfqfkxsjejedqquelbtivkifxhhumixuvugitksjeqjdbqvdpskdurlnoqueqrfsvoakplfpltkhexexhfcepyngjsvwbjhxvioqjoudiitwjzyjbunxpddmsmfjtfzczqhwgeqsqzbqddcjrhyjjuxjqdynnjyfyncxyfnipmkznnutxpcdaebwtmlapcaflggltsdrykukjhgsipfxytvtqrmvlpxgxwzwbhhahdvygfvekfktccojxmehdyvgtucpgumfqdusejwaslxgbbyqonjektuvvpyttirecccpkudvwrxdsmbvqdjlnvlzafhhnhrumuoxqdxlwdjpqpshxywzfwdbgwrotwmqjmmafjqzplkaibphrmgrthwlpvujlgtjsqrthoesyflgijbivexczwmjqegrecjgnbcpbytnhpqjvbeoclgkskpgahfpmyebbtuncmibwvpygemgcacjyalitmpgwefqnrfmzscevwbsleamphucziyydfluonvqatgclxnlhsssjkpbhbynixzqwrcgafgtaabpprczpzsdcliyxlcqrjkzumycpedzycncennkbtzlnnuzlwjxbfzrwoqlfgylgfyrxthesbagezciuuoaqbrbjjlezejbqfkaiayunkydhpctpppsfjnbmklpiyglndyvzhlmpezalcmvxjskdtygqojgpkrssshdxrkvmtwnfulgdwbdnvhwswokrswzprvttxhhozdokvwkmcxktezpbmprqvciipqurtlyfkxjnyuvnkvqzxqffafidpjlocfodyitaadzxb

### **Cálculo de h(M) o hash del mensaje original**

In [104]:
hM = int.from_bytes(hashlib.sha256(msg.encode('utf-8')).digest(), 'big')
hM

87348874898765461166455204892058412222940603411933902983692648625106566196083

### **División y encriptación del mensaje**
---

Aquí Alice toma el mensaje de 1050 caracteres y lo divide en pedazos de 128. Estos luego se transforman a bytes, luego a int y se realiza la encriptación con la llave pública de Bob (nB). Finalmente, cada pedazo se agrega a la lista cryptParts

In [105]:
#divide el mensaje en pedazos de 128 caracteres, los cifra y junta en una lista
cryptParts = []

for i in range(0, len(msg), 128):
  part = msg[i:i+128]
  partBytes = part.encode()
  partInt = int.from_bytes(partBytes, 'big')
  cryptPart = pow(partInt, e, nB)
  cryptParts.append(cryptPart)

cryptParts

[7644179938298978250581888538061132744215251982099206969052923942028320679300490579506367977233991168585912774912661946962576082980056840463232265803495090355345408702183310454076653761238603502400584735017432853655553152250227801130868945269769347268383257285705433381859697893157426698958319651200018114551033589743974292708164310419165581371696250940366006279192901288939286382528594760746064914814683906998869857148606384212072368648421635703372312086127817693210933121887765911873323423516348609440475443546424566222500040047761816926287350068533948247085251853719287707162935457248944431723038070206446563954507,
 10579654630209997319660839826594945523315899782358816817518135492060173896707088961303236163929346213582189022251770431461703164440175284090007503601099579430531710688764620208256218450227276614227726093992685346531471162408707447153674237239021323548477653294008362577185819474446097647581442154408660468629981032211842605999044101517158932612568986942520753239144389666816376075

### **Bob recibe esas porciones del mensaje y las decifra y junta al final como strings para obtener el mensaje**



---

Aquí se revisa cada cryptPart o pedazo encriptado y se realiza su desencriptación. Una vez hecho esto, se convierte a bytes, luego a strings y esta string se añade a la lista decryptParts. Al final esto resulta en el mensaje completo original


In [106]:
decryptParts = []

for cryptPart in cryptParts:
  partInt = pow(cryptPart, dB, nB)
  partBytes = partInt.to_bytes((partInt.bit_length() + 7)//8, 'big')
  partStr = partBytes.decode()
  decryptParts.append(partStr)

decryptMsg = ''.join(decryptParts)
decryptMsg

'kxlkmugukrfpnkanzaacxklfvqifgamefkywjaepgksgulzirezjpncebivuhljziqxwiwtigqwnuvfbmrxfnbnventsrkhbjzvmiidlvmzbruzhdzoneaufkquorvmlsokpmxinaipetdlbraynquimrxsarsszzblpdgvrmaljgbxuqdfqfkxsjejedqquelbtivkifxhhumixuvugitksjeqjdbqvdpskdurlnoqueqrfsvoakplfpltkhexexhfcepyngjsvwbjhxvioqjoudiitwjzyjbunxpddmsmfjtfzczqhwgeqsqzbqddcjrhyjjuxjqdynnjyfyncxyfnipmkznnutxpcdaebwtmlapcaflggltsdrykukjhgsipfxytvtqrmvlpxgxwzwbhhahdvygfvekfktccojxmehdyvgtucpgumfqdusejwaslxgbbyqonjektuvvpyttirecccpkudvwrxdsmbvqdjlnvlzafhhnhrumuoxqdxlwdjpqpshxywzfwdbgwrotwmqjmmafjqzplkaibphrmgrthwlpvujlgtjsqrthoesyflgijbivexczwmjqegrecjgnbcpbytnhpqjvbeoclgkskpgahfpmyebbtuncmibwvpygemgcacjyalitmpgwefqnrfmzscevwbsleamphucziyydfluonvqatgclxnlhsssjkpbhbynixzqwrcgafgtaabpprczpzsdcliyxlcqrjkzumycpedzycncennkbtzlnnuzlwjxbfzrwoqlfgylgfyrxthesbagezciuuoaqbrbjjlezejbqfkaiayunkydhpctpppsfjnbmklpiyglndyvzhlmpezalcmvxjskdtygqojgpkrssshdxrkvmtwnfulgdwbdnvhwswokrswzprvttxhhozdokvwkmcxktezpbmprqvciipqurtlyfkxjnyuvnkvqzxqffafidpjlocfodyitaadzxb

**Calculamos el h(M)' y comparamos con el h(M) del mensaje original para ver si el mensaje se transfirió con éxito a Bob**

In [107]:
hM1 = int.from_bytes(hashlib.sha256(decryptMsg.encode('utf-8')).digest(), 'big')

if(hM == hM1):
  print("El mensaje es auténtico")
else:
  print("El mensaje no es auténtico")

El mensaje es auténtico


## **Código en un solo cuadro**

In [108]:
import Crypto.Util.number
import random
import hashlib
import random
import string

e = 65537

#llave pública de Alice
pA = Crypto.Util.number.getPrime(1024)
qA = Crypto.Util.number.getPrime(1024)
nA = pA * qA

#Llave privada de Alice
phiA = (pA - 1) * (qA - 1)
dA = pow(e, -1, phiA)


#llave pública de Bob
pB = Crypto.Util.number.getPrime(1024)
qB = Crypto.Util.number.getPrime(1024)
nB = pB * qB

#Llave privada de Bob
phiB = (pB - 1) * (qB - 1)
dB = pow(e, -1, phiB)

#generación del mensaje de 1050 caracteres
def generate_message(length):
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(length))

msg = generate_message(1050)

#cálculo del hash del mensaje original
hM = int.from_bytes(hashlib.sha256(msg.encode('utf-8')).digest(), 'big')


#división del mensaje y encriptación con la llave pública de Bob
cryptParts = []

for i in range(0, len(msg), 128):
  part = msg[i:i+128]
  partBytes = part.encode()
  partInt = int.from_bytes(partBytes, 'big')
  cryptPart = pow(partInt, e, nB)
  cryptParts.append(cryptPart)


#desencriptación de cada pedazo de 128 caracteres y unión en un solo mensaje
decryptParts = []

for cryptPart in cryptParts:
  partInt = pow(cryptPart, dB, nB)
  partBytes = partInt.to_bytes((partInt.bit_length() + 7)//8, 'big')
  partStr = partBytes.decode()
  decryptParts.append(partStr)

decryptMsg = ''.join(decryptParts)

#cálculo de hash del mensaje recibido por Bob y verificación de autenticidad
hM1 = int.from_bytes(hashlib.sha256(decryptMsg.encode('utf-8')).digest(), 'big')

if(hM == hM1):
  print("El mensaje es auténtico")
else:
  print("El mensaje no es auténtico")

El mensaje es auténtico
